In [22]:
import torch
import imageio.v3 as iio
import matplotlib.pyplot as plt
import numpy as np
import cv2
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas


In [11]:
device = 'cuda:0'
cotracker = torch.hub.load("facebookresearch/co-tracker", "cotracker2_online").to(device)

Using cache found in /home/uichan8/.cache/torch/hub/facebookresearch_co-tracker_main


In [12]:
video_path = 'image/sample.mp4'
frames = iio.imread(video_path)
grid_size = 20
video = torch.tensor(frames).permute(0, 3, 1, 2)[None].float().to(device)
video.shape

torch.Size([1, 406, 3, 720, 1280])

In [13]:
cotracker(video_chunk=video, is_first_step=True, grid_size=grid_size)
for ind in range(0, video.shape[1] - cotracker.step, cotracker.step):
    pred_tracks, pred_visibility = cotracker(
        video_chunk=video[:, ind : ind + cotracker.step * 2]
    )

In [14]:
pred_tracks.shape

torch.Size([1, 406, 400, 2])

In [15]:
pred_visibility.shape

torch.Size([1, 406, 400])

In [26]:
# 동영상 저장 설정
frame_width = frames.shape[1]
frame_height = frames.shape[0]
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # 코덱을 XVID에서 mp4v로 변경
out = cv2.VideoWriter('output_video.mp4', fourcc, 30.0, (frame_width, frame_height))

# 프레임 생성 및 저장 루프
for i in range(pred_tracks.shape[1]):
    target_image = video[0, i, :, :, :]
    target_image = target_image.permute(1, 2, 0)
    target_image = target_image.cpu().numpy().astype(np.uint8)
    target_point = pred_tracks[0, i, :, :].cpu().numpy()
    target_valid = pred_visibility[0, i, :].cpu().numpy()

    # Matplotlib을 사용하여 이미지 생성
    fig, ax = plt.subplots()
    ax.imshow(target_image)
    ax.scatter(target_point[:, 0] * target_valid, target_point[:, 1] * target_valid, c='r', s=1)
    
    # figure를 frame으로 변환
    canvas = FigureCanvas(fig)
    canvas.draw()
    frame = np.frombuffer(canvas.buffer_rgba(), dtype=np.uint8)
    frame = frame.reshape(canvas.get_width_height()[::-1] + (4,))
    
    # frame을 BGR로 변환 후 동영상에 저장
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    out.write(frame)
    
    # 플롯 닫기
    plt.close(fig)

out.release()
    

/tmp/ipykernel_308603/801931199.py:23: MatplotlibDeprecationWarning: The tostring_rgb function was deprecated in Matplotlib 3.8 and will be removed in 3.10. Use buffer_rgba instead.
  frame = np.frombuffer(canvas.tostring_rgb(), dtype=np.uint8)
